In [1]:
!pip install numpy~=1.0  # 1.0 instead of 2.0 for pymatching compatibility later
!pip install scipy
!pip install stim~=1.14
!pip install pymatching~=2.0

In [2]:
import stim
print(stim.__version__)

1.14.0


In [3]:
import stim
import pymatching
import numpy as np
def count_logical_errors(circuit: stim.Circuit, num_shots: int) -> int:
    # Sample the circuit.
    sampler = circuit.compile_detector_sampler()
    detection_events, observable_flips = sampler.sample(num_shots, separate_observables=True)

    # Configure a decoder using the circuit.
    detector_error_model = circuit.detector_error_model(decompose_errors=True)
    matcher = pymatching.Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events)

    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot]
        predicted_for_shot = predictions[shot]
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors

In [4]:
import stim
def get_logical_err_rate(distance: int, rounds: int, phy_err_p: int, num_shots: int) -> float:
    # Generate Surface Code circuit with the specified parameters.
    circuit = stim.Circuit.generated(
            "surface_code:rotated_memory_z",
            rounds=rounds,
            distance=distance,
            after_clifford_depolarization=phy_err_p,
            after_reset_flip_probability=phy_err_p,
            before_measure_flip_probability=phy_err_p,
            before_round_data_depolarization=phy_err_p,
        )
    return count_logical_errors(circuit, num_shots) / num_shots

In [5]:
print(get_logical_err_rate(distance=7, rounds=21, phy_err_p=0.001, num_shots=100000))

8e-05


In [6]:
class PhysicalQubit:
    def __init__(self, gate_error_rate, measurement_error_rate, 
                 single_qubit_gate_time=50e-9, two_qubit_gate_time=100e-9):
        self.single_qubit_gate_time = single_qubit_gate_time
        self.two_qubit_gate_time = two_qubit_gate_time      
        self.gate_error_rate = gate_error_rate              
        self.measurement_error_rate = measurement_error_rate

In [7]:
class SurfaceCode:
    def __init__(self, code_distance, physical_qubit_params):
        self.code_distance = code_distance
        self.physical_qubit = physical_qubit_params

    def logical_error_rate(self):
        p_physical = self.physical_qubit.gate_error_rate
        return p_physical ** (self.code_distance / 2)

    def physical_qubits_per_logical(self):
        return 2 * self.code_distance ** 2

In [8]:
class TFactory:
    def __init__(self, surface_code, distillation_level=1):
        self.surface_code = surface_code
        self.distillation_level = distillation_level

    def required_physical_qubits(self):
        return 15 * self.surface_code.physical_qubits_per_logical() 

    def distillation_time(self):
        return 10 * self.surface_code.code_distance * self.physical_qubit.two_qubit_gate_time

In [9]:
class Algorithm:
    def __init__(self, t_gates, cnot_gates, total_cycles):
        self.t_gates = t_gates         
        self.cnot_gates = cnot_gates    
        self.total_cycles = total_cycles  

In [10]:
class ResourceEstimator:
    def __init__(self, algorithm, surface_code, t_factory):
        self.algorithm = algorithm
        self.surface_code = surface_code
        self.t_factory = t_factory

    def estimate(self):
        logical_qubits = self.algorithm.cnot_gates * 2

        physical_per_logical = self.surface_code.physical_qubits_per_logical()
        t_factory_qubits = self.t_factory.required_physical_qubits()

        total_physical = (logical_qubits * physical_per_logical) + t_factory_qubits

        logical_cycle_time = self.surface_code.code_distance * self.surface_code.physical_qubit.two_qubit_gate_time
        total_time = self.algorithm.total_cycles * logical_cycle_time

        return {
            "logical_qubits": logical_qubits,
            "physical_qubits": total_physical,
            "total_time_seconds": total_time
        }

In [14]:
physical_qubit = PhysicalQubit(gate_error_rate=1e-3, measurement_error_rate=1e-2)

surface_code = SurfaceCode(code_distance=5, physical_qubit_params=physical_qubit)

t_factory = TFactory(surface_code=surface_code, distillation_level=1)

algorithm = Algorithm(t_gates=1e6, cnot_gates=5e5, total_cycles=1e10)

estimator = ResourceEstimator(algorithm, surface_code, t_factory)
result = estimator.estimate()

print(f"logical qubit num: {result['logical_qubits']}")
print(f"physical qubit num: {result['physical_qubits']}")
print(f"runtime: {result['total_time_seconds']/3600:.2f} hour")

logical qubit num: 1000000.0
physical qubit num: 50000750.0
runtime: 1.39 hour
